In [2]:
%load_ext autoreload
%autoreload 2

In [18]:
import os, sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
# sys.path.append('../../satclip/satclip/')
# from load import get_satclip
sys.path.append('../scripts/')
from loadpaths_glc import loadpaths
import data_loading_utils as dlu
import glc_prediction_utils as gpu
path_dict = loadpaths()
import h3pandas

In [17]:
dict_dfs, dict_dfs_train = dlu.load_metadata(create_geo=False)

Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2']


In [5]:
for key, df in dict_dfs.items():
    print('\n----')
    print(f'{key}: {df.shape[0]} samples')
    print(df.columns)
    n_unique_samples = df['surveyId'].nunique()
    if 'test' not in key:
        n_unique_species = df['speciesId'].nunique()
    else:
        n_unique_species = 'NA'
    print(f'Unique samples: {n_unique_samples}')
    print(f'Unique species: {n_unique_species}')


----
df_train_pa: 1483637 samples
Index(['lon', 'lat', 'year', 'region', 'country', 'speciesId', 'surveyId',
       'geometry'],
      dtype='object')
Unique samples: 88987
Unique species: 5016

----
df_test_pa: 4716 samples
Index(['lon', 'lat', 'year', 'region', 'country', 'surveyId', 'geometry'], dtype='object')
Unique samples: 4716
Unique species: NA


In [6]:
tmp = dlu.get_path_sat_patch_per_survey(1400102)
tmp

('/Users/t.vanderplas/data_offline/data_geolifeclef2024/geolifeclef-2024/PA_Train_SatellitePatches_RGB/pa_train_patches_rgb/02/01/1400102.jpeg',
 '/Users/t.vanderplas/data_offline/data_geolifeclef2024/geolifeclef-2024/PA_Train_SatellitePatches_NIR/pa_train_patches_nir/02/01/1400102.jpeg')

In [7]:
tmp = dlu.load_multiple_env_raster()
tmp

,surveyId,Elevation,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,...,Bio10,Bio11,Bio12,Bio13,Bio14,Bio15,Bio16,Bio17,Bio18,Bio19
0,212,47.0,8.0,140.0,214.0,151.0,292.0,159.0,73.0,284.0,...,2958,2816,6355,933,143,395,2282,770,770,1917
1,222,6.0,12.0,120.0,225.0,101.0,94.0,379.0,58.0,650.0,...,2890,2746,7286,792,376,201,2283,1336,2076,1653
2,243,10.0,5.0,115.0,261.0,86.0,131.0,373.0,54.0,750.0,...,2888,2759,8546,1117,372,295,2986,1438,2448,1838
3,324,116.0,14.0,118.0,179.0,117.0,225.0,230.0,57.0,238.0,...,2938,2802,10719,1163,564,197,3216,1948,1948,2926
4,333,84.0,14.0,138.0,233.0,176.0,328.0,250.0,71.0,231.0,...,2927,2792,9353,1095,503,249,3186,1638,1638,2677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88982,3919553,23.0,12.0,116.0,199.0,65.0,116.0,311.0,59.0,622.0,...,2887,2752,8000,919,408,218,2505,1425,2187,1723
88983,3919592,138.0,1.0,118.0,245.0,113.0,156.0,405.0,54.0,555.0,...,2888,2748,6932,781,314,240,2242,1214,2029,1498
88984,3919620,37.0,12.0,113.0,233.0,70.0,71.0,253.0,56.0,798.0,...,2887,2746,9745,1201,440,246,3167,1718,2489,2413
88985,3919640,131.0,1.0,130.0,162.0,99.0,55.0,234.0,50.0,803.0,...,2895,2722,7301,816,390,211,2357,1336,2357,1558


## geo

In [83]:
dict_dfs, dict_dfs_species = dlu.load_metadata(create_geo=True, add_h3=8)
# type(dict_dfs['df_test_pa'])

Dropped PO data because takes ages with geometry
Columns dropped: ['taxonRank', 'geoUncertaintyInM', 'date', 'areaInM2', 'publisher', 'month', 'day', 'region', 'dayOfYear', 'country']


In [84]:
dict_dfs['df_test_pa'].head()

,h3_08,lng,lat,year,surveyId,geometry
0,881f246e4dfffff,10.033550,57.12081,2019,642,POINT (10.03355 57.12081)
1,881f9c0731fffff,7.333000,46.22997,2019,1792,POINT (7.33300 46.22997)
2,88396226adfffff,1.843658,42.58006,2018,3256,POINT (1.84366 42.58006)
3,881f8b7b15fffff,11.720090,46.26149,2021,3855,POINT (11.72009 46.26149)
4,881f22cf49fffff,9.361870,55.90245,2017,4889,POINT (9.36187 55.90245)


In [85]:
dict_dfs['df_train_po']

KeyError: 'df_train_po'

In [107]:
dict_dfs['df_train_pa'].sindex.intersection(dict_dfs['df_test_pa'].geometry[3].buffer(.1).bounds)

array([24974, 53072, 32072, 18534, 43387, 74353, 59721, 78450, 21422,
        6029, 46582,   259, 66332, 65534,  9488, 85224, 31854, 15551,
       68209, 47067, 75699, 88447,  7589, 64797, 13908, 57059, 16656,
       52473, 40783,  4761, 15432,  8999, 80429])

In [105]:
dict_dfs['df_test_pa'].geometry[0].buffer(1).bounds

(9.03355, 56.12081, 11.03355, 58.12081)

In [116]:
dlu.load_multiple_env_raster(mode='test').head()

,surveyId,Elevation,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,...,Bio10,Bio11,Bio12,Bio13,Bio14,Bio15,Bio16,Bio17,Bio18,Bio19
0,642,24.0,9.0,114.0,229.0,96.0,95.0,389.0,48.0,634.0,...,2890,2745,8165,896,425,210,2555,1461,2358,1832
1,1792,727.0,10.0,126.0,204.0,156.0,238.0,259.0,68.0,349.0,...,2922,2725,7694,780,499,140,2276,1635,2276,1929
2,3256,2381.0,10.0,109.0,246.0,193.0,210.0,547.0,59.0,370.0,...,2817,2660,10128,1068,585,146,2995,2179,2465,2314
3,3855,2367.0,10.0,112.0,252.0,195.0,189.0,489.0,60.0,375.0,...,2829,2645,15348,1981,488,395,5699,1822,5541,1825
4,4889,109.0,5.0,113.0,146.0,86.0,96.0,220.0,53.0,751.0,...,2882,2740,9387,1110,443,215,2930,1747,2380,2370


In [175]:
tmp = gpu.predict_using_buffer(dict_dfs=dict_dfs, dict_dfs_species=dict_dfs_species,
                            save_pred=True, method='top_25', buffer_deg=0.5)


100%|██████████| 4716/4716 [00:26<00:00, 179.16it/s]


Saved predictions to: /Users/t.vanderplas/data_offline/data_geolifeclef2024/predictions_glc2024/GLC24_vdplasthijs_predictions-buffer-lc-0.5-top_25_2024-04-15_21-05.csv


In [49]:
dict_dfs['df_train_pa'].head()

,h3_08,lng,lat,year,region,country,speciesId,surveyId
0,883963a4e7fffff,3.099038,43.134956,2021,MEDITERRANEAN,France,6874.0,212
1,883963a4e7fffff,3.099038,43.134956,2021,MEDITERRANEAN,France,476.0,212
2,883963a4e7fffff,3.099038,43.134956,2021,MEDITERRANEAN,France,11157.0,212
3,883963a4e7fffff,3.099038,43.134956,2021,MEDITERRANEAN,France,8784.0,212
4,883963a4e7fffff,3.099038,43.134956,2021,MEDITERRANEAN,France,4530.0,212
